Daten einlesen
--------------

In [1]:
from sklearn.datasets import load_iris
data = load_iris()

In [2]:
import pandas as pd

In [3]:
pd.DataFrame(data["data"])

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


Daten splitten
--------------

In [4]:
X = data.data
y = data.target

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3)

In [6]:
print('X Train: {}'.format(X_train.shape))

X Train: (100, 4)


In [7]:
print('Y Test: {}'.format(y_test.shape))

Y Test: (50,)


Decision Tree Classifier - initialisieren und trainieren
---------

In [8]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(max_depth=5, random_state=0)
dt = dt.fit(X_train,y_train)

Decision Tree Classifier - Modell testen
---------

In [9]:
score = dt.score(X_test,y_test)
print('Decision Tree scores with {}% accuracy'.format(score*100))
#dt.feature_importances_

Decision Tree scores with 96.0% accuracy


In [10]:
from sklearn.tree import export_graphviz

attribute_names = ['sepal length in cm','sepal width in cm','petal length in cm','petal width in cm', 'class']

#Export as dot file
export_graphviz(dt, out_file='iris_5.dot', class_names = True, feature_names = attribute_names[0:4])

#Export dot to png 
from subprocess import check_call
check_call(['dot','-Tpng','iris_5.dot','-o','iris_5.png'])

0

Random Forest Classifier - initialisieren und trainieren
-------

In [11]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(max_depth=5, random_state=0)
rf.fit(X_train,y_train)

RandomForestClassifier(max_depth=5, random_state=0)

Random Forest Classifier - Modell testen
-----------

In [12]:
score = rf.score(X_test,y_test)
print('Random Forest scores with {}% accuracy'.format(score*100))

Random Forest scores with 98.0% accuracy


Random Forest Classifier Limebit - Modell testen
-----

In [13]:
#RandomForestClassifier object
random_forest_classifier = RandomForestClassifier(n_estimators=10)

In [14]:
from sklearn.model_selection import cross_val_score, KFold
#list with accuracies with different test and training sets of Random Forest
k_fold = KFold(n_splits=5, shuffle=True, random_state=0)
accuracies_rand_forest = cross_val_score(random_forest_classifier, X_test, y_test, cv=k_fold, n_jobs=1)

In [15]:
from sklearn.model_selection import GridSearchCV

#tree parameters which shall be tested
tree_para = {'criterion':['gini','entropy'],'max_depth':[i for i in range(1,20)]} #, 'min_samples_split':[i for i in range (2,20)]}

#GridSearchCV object
grd_clf = GridSearchCV(dt, tree_para, cv=5)

#creates differnt trees with all the differnet parameters out of our data
grd_clf.fit(X_train, y_train)

#best paramters that were found
best_parameters = grd_clf.best_params_  
print(best_parameters)  

{'criterion': 'gini', 'max_depth': 3}


In [16]:
#new tree object with best parameters
model_with_best_tree_parameters = grd_clf.best_estimator_

#k_fold object
k_fold = KFold(n_splits=5, shuffle=True, random_state=0)

#scores reached with different splits of training/test data 
k_fold_scores = cross_val_score(model_with_best_tree_parameters, X_test, y_test, cv=k_fold, n_jobs=1)

In [17]:
import numpy as np

#scores reached with different splits of training/test data 
k_fold_scores = cross_val_score(dt, X_test, y_test, cv=k_fold, n_jobs=1)

#arithmetic mean of accuracy scores 
mean_accuracy = np.mean(k_fold_scores)

#arithmetic mean of accuracy scores 
mean_accuracy_best_parameters_tree = np.mean(k_fold_scores)

##arithmetic mean of the list with the accuracies of the Random Forest
accuracy_rand = np.mean(accuracies_rand_forest)

print('Accuracy Random Forest ' + str(round(accuracy_rand,4)))
print('Old accuracy: ' + str(round(mean_accuracy,4)))
print('Best tree accuracy: ' + str(round(mean_accuracy_best_parameters_tree,4)))

Accuracy Random Forest 0.94
Old accuracy: 0.92
Best tree accuracy: 0.92
